## BRONZE TO SILVER LAYER

### GOLD LAYER - PROCESS DIVIDEND HISTORY

### Description
The notebook is designed to process stock dividend data by merging it with holdings data, calculating dividend amounts, and saving the result to a CSV file. The function follows these steps:

In [ ]:
# Importing Common Utility Funcation

import pandas as pd
from PortfolioTracker import GlobalPath

In [ ]:
# Import necessary libraries and utility functions
%run ../COMMON/common_utility.ipynb

In [ ]:
# Instantiate GlobalPath
holdingshistory_gold_file_path = GlobalPath(
    "DATA/GOLD/Holdings/HoldingsHistory_data.csv"
)
stockevents_silver_file_path = GlobalPath(
    "DATA/SILVER/StockEvents/StockEvents_data.csv"
)
dividend_gold_file_path = GlobalPath("DATA/GOLD/Dividend/Dividend_data.csv")

In [ ]:
# Load holdings data from the GOLD layer
df_holdings = pd.read_csv(holdingshistory_gold_file_path)
df_holdings["date"] = pd.to_datetime(df_holdings["date"])
print(f"Loaded GOLD Layer holdings data from: {holdingshistory_gold_file_path}")

In [ ]:
# Load dividend data from the SILVER layer
df_dividends = pd.read_csv(stockevents_silver_file_path)
df_dividends["date"] = pd.to_datetime(df_dividends["date"])
print(
    f"Loaded SILVER Layer stock dividend data from: {stockevents_silver_file_path}"
)

In [ ]:
# Filter for dividend events only
df_dividends = df_dividends[df_dividends["event"].str.upper() == "DIVIDENDS"]

# Merge dividend data with holdings data
df_dividend = pd.merge(
    df_holdings, df_dividends, on=["date", "symbol"], how="left"
)

# Calculate the dividend amount
df_dividend["dividend_amount"] = (
    df_dividend["value"].fillna(0) * df_dividend["holding_quantity"]
)
df_dividend["dividend_amount"] = df_dividend["dividend_amount"].round(2)

# Filter out rows where dividend amount is 0
df_dividend = df_dividend[df_dividend["dividend_amount"] != 0]

In [ ]:
# Sort and format the DataFrame
df_dividend = df_dividend.sort_values(
    by=["date", "segment", "symbol"]
).reset_index(drop=True)
df_dividend = df_dividend[["date", "segment", "symbol", "dividend_amount"]]

# Save the result to a new CSV file
df_dividend.to_csv(dividend_gold_file_path, index=False)
print(
    f"GOLD Layer CSV file for Holdings successfully created at: {dividend_gold_file_path}"
)

# Display DataFrame debugrmation
df_dividend.info()